## Data Wrangling Challenge

In [3]:
import pandas as pd
mypath = "/Users/jurajkapasny/Drive/Data/Car_crashes/"

In [16]:
data = pd.read_csv("~/Downloads/VT_VehicleAnimal_Collisions__2006.csv", 
                   na_values=['', ' '],
                   parse_dates=["DATE_"]
                  )
data["MONTH_"] = data.DATE_.dt.month
data.dropna(subset=['X', 'Y'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Y'].astype(str) + ',' + data['X'].astype(str)
print(data.shape)
data.head()

(2244, 23)


,X,Y,OBJECTID,MSRI_CODE,MSRI_DESCR,DATE_,TOWN_ID,ROUTE_DES,ROUTE,BEGIN_MM,...,TOWN,RT_TOWN,RT_NUM,DAY_,MONTH_,YEAR_,REP_AGEN,LOCATION,YEAR_INT,ll
4,-72.989424,42.833309,5,Deer,Deer,2004-04-16 00:00:00+00:00,209.0,VT 8,008,0.35,...,READSBORO,NaN,NaN,NaN,4.0,2004,AOT,NaN,2004,"42.83330874047745,-72.98942388725152"
5,-72.375495,43.581005,6,Deer RK AB,"Deer, RoadKill, Adult Buck",2005-03-22 00:00:00+00:00,1409.0,US 5,005,5.00,...,HARTLAND,NaN,NaN,NaN,3.0,2005,AOT,NaN,2005,"43.581005107295645,-72.37549549789222"
6,-72.840257,43.235023,7,Lg Bird,"Large Bird (hawk, owl, turkey, waterfowl)",2004-11-15 00:00:00+00:00,1310.0,VT 11,011,0.50,...,LONDONDERRY,NaN,NaN,NaN,11.0,2004,AOT,NaN,2004,"43.23502288333559,-72.8402568531839"
7,-72.830991,44.783002,8,Otter,Otter,2004-10-18 00:00:00+00:00,601.0,VT 36,036,1.00,...,BAKERSFIELD,NaN,NaN,NaN,10.0,2004,AOT,NaN,2004,"44.78300220380285,-72.83099079061695"
9,-72.514595,44.242323,10,Beaver,Beaver,2005-04-22 00:00:00+00:00,1207.0,US 2,002,0.19,...,EAST MONTPEL,NaN,NaN,NaN,4.0,2005,AOT,NaN,2005,"44.24232301744294,-72.51459452575021"


In [17]:
#set the keys
foursquare_id = "5W1M2W2A3IXWCYC23S44DKA4AXVG3LQPT5JCZ4TYWYN0OO5T"
foursquare_secret = "HNZVVCWLJ1C1J4C4AZCJWDLNEZZ1HBE5XVLIIIVBZSQN0VR4"

In [19]:
from foursquare import Foursquare

In [20]:
client = Foursquare(client_id = foursquare_id,
                    client_secret = foursquare_secret)

In [26]:
import time
number_of_bars = []
ind = 0
for crash in data.iterrows():
    if ind % 100 == 0:
        print(ind)
    try:
        this_crash = crash[1]
        county = this_crash["COUNTY_NAME"] + ",MD"
        response = client.venues.search({"near" : county,
                                         "limit" : 50,
                                         "intent" : "browse",
                                         "radius" : 5000,
                                         "categoryId" : "4d4b7105d754a06376d81259"})
        #print response
        number_of_bars.append(len(response["venues"]))
    except Exception as e:
        print(e)
        if e == "Quota exceeded":
            print("exceeded quota: waiting for an hour")
            time.sleep(3600)
        number_of_bars.append(-1)
#     break
    ind += 1
data["number_of_bars"] = number_of_bars

0
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COUNTY_NAME'
'COU

In [27]:
number_of_bars

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,


In [ ]:
#dark sky

In [ ]:
import requests
import time
api_key = "ece26d6a4766b34892c5e71df0d5bc1e"

weather = {}
ind = 0
for crash in data.iterrows():
    if ind % 100 == 0:
        print ind,
    try:
        this_acc = crash[1]
        lat = this_acc["LAT"]
        lng = this_acc["LNG"]
        tm = this_acc["ACC_DATE_ACC_TIME"]
        
        base_url = "https://api.darksky.net/forecast/%s/%s,%s,%s" %(api_key, lat, lng, str(tm).replace(" ", "T"))
        headers = {"User-agent": "curl/7.43.0",
                   "Accept": "application/json"}
        params = {
            "timestamp" : tm,
            "units" : "si",
        }
        
        response = requests.get(base_url,
                        params=params,
                        headers=headers)
        if response.status_code == 200:
            weather[crash[0]] = response.json()["currently"]
        else:
            weather[crash[0]] = {}
            print response.reason
    except Exception as e:
        print e
        weather[crash[0]] = {}
    ind += 1


In [ ]:
data = pd.concat([data, pd.DataFrame(weather).transpose()], axis=1)